# Proxy
- http://free-proxy.cz/zh/proxylist/country/US/https/ping/all
- https://www.us-proxy.org/

# 安裝與載入套件

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [817 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restric

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=options)

# 收集IP清單

In [ ]:
IPPool = []
for i in range(1,6):
    url = 'http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/{}'.format(i)
    print('Dealing with {}'.format(url))
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    for j in soup.select('tbody > tr'):
        if re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', str(j)):
            IP = re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', str(j))[0]
            Port = re.findall('class="fport" style="">(.*?)</span>', str(j))[0]
            IPPool.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
    print('There are {} IPs in Pool'.format(len(IPPool)))
IPPool = pd.concat(IPPool, ignore_index=True)
IPPool

Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/1
There are 35 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/2
There are 70 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/3
There are 104 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/4
There are 104 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/5
There are 104 IPs in Pool


,IP,Port
0,159.89.245.69,3128
1,138.197.5.192,8888
2,38.91.100.122,3128
3,155.138.130.39,8000
4,198.98.55.168,8080
...,...,...
99,206.189.151.212,8000
100,104.248.107.151,3128
101,165.227.114.142,8080
102,3.94.59.144,3128


# 測試並保留有效的IP

In [ ]:
ActIps = []
for IP, Port in zip(IPPool['IP'],IPPool['Port']):
    proxy = {'http':'http://'+ IP + ':' + Port,
             'https':'https://'+ IP + ':' + Port} 
    try:
        # 隨機找的一篇新聞
        url = 'https://www.chinatimes.com/realtimenews/20200205004069-260408'
        resp = requests.get(url, proxies=proxy, timeout=2)
        if str(resp.status_code) == '200':
            ActIps.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
            print('Succed: {}:{}'.format(IP, Port))
        else:
            print('Failed: {}:{}'.format(IP, Port))
    except:
            print('Failed: {}:{}'.format(IP, Port))
ActIps = pd.concat(ActIps, ignore_index=True)
ActIps

Failed: 159.89.245.69:3128
Failed: 138.197.5.192:8888
Succed: 38.91.100.122:3128
Failed: 155.138.130.39:8000
Failed: 198.98.55.168:8080
Failed: 172.105.97.190:8080
Succed: 38.91.107.242:3128
Failed: 198.98.58.178:8080
Failed: 64.227.14.30:8080
Succed: 198.98.56.71:8080
Succed: 104.216.21.114:80
Failed: 159.203.207.56:3128
Failed: 104.5.204.218:8080
Failed: 205.185.115.100:8080
Failed: 206.189.113.118:8080
Failed: 38.91.107.117:3128
Failed: 107.178.6.30:8080
Failed: 44.230.90.14:8888
Failed: 209.97.189.98:8080
Failed: 159.203.87.130:3128
Failed: 68.183.188.100:3128
Failed: 157.245.146.75:8080
Failed: 165.22.57.92:8080
Succed: 167.71.213.19:8080
Failed: 165.227.215.214:80
Succed: 198.98.50.164:8080
Failed: 35.247.229.191:3128
Succed: 64.227.6.252:8080
Failed: 167.71.213.125:8080
Failed: 167.71.213.179:8080
Failed: 157.230.244.46:8080
Failed: 68.15.31.118:48678
Succed: 35.240.158.133:80
Failed: 35.240.173.246:80
Failed: 18.163.152.238:3128
Failed: 136.25.2.43:40614
Failed: 167.71.202.40:8

,IP,Port
0,38.91.100.122,3128
1,38.91.107.242,3128
2,198.98.56.71,8080
3,104.216.21.114,80
4,167.71.213.19,8080
5,198.98.50.164,8080
6,64.227.6.252,8080
7,35.240.158.133,80
8,52.124.6.146,40834
9,140.144.90.53,80
